In [0]:
try: 
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

print("Version: ", tf.version)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

TensorFlow is already loaded. Please restart the runtime to change versions.
Version:  <module 'tensorflow._api.v1.version' from '/usr/local/lib/python3.6/dist-packages/tensorflow_core/_api/v1/version/__init__.py'>
Eager mode:  False
GPU is available


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import os
import cv2
import glob
import shutil
from tensorflow.keras import optimizers
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, BatchNormalization
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings("ignore")

In [0]:
train_data_dir = '/content/drive/My Drive/Thinkful Datasets/final_images/train/'
test_data_dir = '/content/drive/My Drive/Thinkful Datasets/final_images/test/'

In [0]:
img_width, img_height = 250, 250
batch_size = 40

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [0]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.3,
    zoom_range=.1,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255,)

In [0]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    #shuffle=False,
    class_mode='binary')

Found 616 images belonging to 2 classes.
Found 264 images belonging to 2 classes.


In [0]:
train_len = 616
test_len = 264

In [0]:
from tensorflow.keras.layers import Input, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Flatten
from tensorflow.keras import Model

In [0]:
inputs = Input(shape=(250, 250, 3))
normalization = BatchNormalization()(inputs)
conv2d = Conv2D(32, kernel_size=(7,7))(normalization)
conv2d_1 = Conv2D(16, kernel_size=(3,3))(conv2d)
max_pooling2d = MaxPooling2D()(conv2d_1)
conv2d_2 = Conv2D(16, kernel_size=(3,3))(max_pooling2d)
conv2d_3 = Conv2D(8, kernel_size=(3,3))(conv2d_2)
max_pooling2d_1 = MaxPooling2D()(conv2d_3)
conv2d_4 = Conv2D(16, kernel_size=(3,3))(max_pooling2d_1)
conv2d_5 = Conv2D(32, kernel_size=(3,3))(conv2d_4)
max_pooling2d_2 = MaxPooling2D()(conv2d_5)
global_average_pooling2d = GlobalAveragePooling2D()(max_pooling2d_2)
dropout_1 = Dropout(0.5)(global_average_pooling2d)
flatten = Flatten()(dropout_1)
dense = Dense(1, activation='sigmoid')(flatten)

In [0]:
model = Model(inputs = inputs, outputs = dense)
model.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 250, 250, 3)]     0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 250, 250, 3)       12        
_________________________________________________________________
conv2d_60 (Conv2D)           (None, 244, 244, 32)      4736      
_________________________________________________________________
conv2d_61 (Conv2D)           (None, 242, 242, 16)      4624      
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 121, 121, 16)      0         
_________________________________________________________________
conv2d_62 (Conv2D)           (None, 119, 119, 16)      2320      
_________________________________________________________________
conv2d_63 (Conv2D)           (None, 117, 117, 8)       1160

In [0]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [0]:
model.fit(
    train_generator,
    steps_per_epoch = 15,
    epochs=50,
    validation_data = validation_generator,
    validation_steps = 5)

Epoch 1/50
15/15 [==============================] - 20s 1s/step - loss: 0.6900 - acc: 0.5590 - val_loss: 0.7036 - val_acc: 0.5050
Epoch 2/50
15/15 [==============================] - 20s 1s/step - loss: 0.6756 - acc: 0.5747 - val_loss: 0.7225 - val_acc: 0.5050
Epoch 3/50
15/15 [==============================] - 19s 1s/step - loss: 0.6729 - acc: 0.6094 - val_loss: 0.6905 - val_acc: 0.5100
Epoch 4/50
15/15 [==============================] - 18s 1s/step - loss: 0.6574 - acc: 0.6094 - val_loss: 0.7650 - val_acc: 0.5050
Epoch 5/50
15/15 [==============================] - 18s 1s/step - loss: 0.6707 - acc: 0.6337 - val_loss: 0.6913 - val_acc: 0.5050
Epoch 6/50
15/15 [==============================] - 19s 1s/step - loss: 0.6678 - acc: 0.6250 - val_loss: 0.6894 - val_acc: 0.5300
Epoch 7/50
15/15 [==============================] - 18s 1s/step - loss: 0.6599 - acc: 0.6178 - val_loss: 0.6989 - val_acc: 0.5100
Epoch 8/50
15/15 [==============================] - 19s 1s/step - loss: 0.6504 - acc: 0.63


1. first model iteration: validation: .6979 .5606 train: .6473 .6445
2. add dropout .1: validation: .7009 .5530 train: .6487 .6461
3. remove dropout, conv2d kernel size 7,7: validation: .7037 .5455 train: .6492 .6494
4. conv2d_3 16 to 8, conv2d_4 32 to 16: validation: .6949 .5455 train: .6543 .6494

In [0]:
model.evaluate(validation_generator)

7/7 [==============================] - 5s 654ms/step - loss: 0.7060 - acc: 0.5379


[0.7059636541775295, 0.5378788]

In [0]:
model.evaluate(train_generator)

16/16 [==============================] - 17s 1s/step - loss: 0.6500 - acc: 0.6461


[0.6500324867665768, 0.6461039]